In [1]:
import pandas as pd
from cuotasimss import CuotasImss
from imss import CalculoIMSSObrero, CalculoIMSSPatron
from isr import ISR
from datetime import datetime
from sbc import SBC
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
df = pd.read_excel('./data/plantilla_laboral2.xlsx', sheet_name='Hoja1')
df.head()

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso
0,1,Martha Rosa Alvarez Martínez,"33,256.68",1500,2019-06-03
1,2,Tania Monserrat Bañuelos Murillo,"25,931.50",1297,2019-06-17
2,6,Karla Berenice Cano Garcia,"22,960.82",1148,2019-07-23
3,7,Jonathan Covarrubias Sustaita,"21,719.58",1086,2019-03-04
4,8,Miguel Dionicio Luna,"27,315.62",1366,2018-01-09


In [3]:
df['Fecha Ingreso'] = pd.to_datetime(df['Fecha Ingreso'], format='%Y-%m-%d')

In [4]:
date_2022 = pd.to_datetime('2022/12/31')

In [5]:
df['SDI_2022'] = df.apply(lambda x: SBC.funcion_sdi(sueldo = x['Sueldo mensual'], a = x['Fecha Ingreso'], b = date_2022), axis=1)
df['SBC_2022'] = df.apply(lambda x: SBC.funcion_sbc(sueldo = x['Sueldo mensual'], vales = x['Vales'], a = x['Fecha Ingreso'], b = date_2022), axis=1)
df['ISR_2022'] = (df['SDI_2022'] * 30.4).apply(lambda x: ISR.function_isr(x))
df['IMSS_O_2022'] = df['SBC_2022'].apply(lambda x: CalculoIMSSObrero.function_imss_obrero(x))
df['IMSS_P_2022'] = df.apply(lambda x: CalculoIMSSPatron.function_imss_patronal(x = x['SBC_2022'], date=date_2022 ), axis=1)
df['ISN_2022'] = df['Sueldo mensual'] * 0.03
_isr_ = df['Sueldo mensual'].apply(lambda x: ISR.function_isr(x))
df['Neto'] = df['Sueldo mensual'] - _isr_ - df['IMSS_O_2022']
df['Aguinaldo_2022'] = df.apply(lambda x: SBC.funcion_aguinaldo(sueldo = x['Sueldo mensual'], a = x['Fecha Ingreso'], b = date_2022), axis=1)
df['Prima Vacacional_2022'] = df.apply(lambda x: SBC.funcion_pv(sueldo = x['Sueldo mensual'], date_ingreso = x['Fecha Ingreso'], date_base = date_2022), axis=1)

In [6]:
df.tail()

,Id,Nombre,Sueldo mensual,Vales,Fecha Ingreso,SDI_2022,SBC_2022,ISR_2022,IMSS_O_2022,IMSS_P_2022,ISN_2022,Neto,Aguinaldo_2022,Prima Vacacional_2022
49,65,José Alfredo López Madrigal,"17,687.64",884,2022-06-16,616.24,616.24,"2,560.34",484.76,"3,544.40",530.63,"14,865.96","4,775.66",0.00
50,66,Jesus Geovanni Ruvalcaba Mota,"17,687.64",884,2022-07-01,616.24,616.24,"2,560.34",484.76,"3,544.40",530.63,"14,865.96","4,421.91",0.00
51,67,Luis Manuel Guerrero Alcantara,"19,500.08",975,2022-08-22,679.38,679.38,"2,970.37",538.03,"3,856.34",585.00,"16,238.00","3,510.01",0.00
52,68,Shaila Barradas Santiago,"18,616.36",931,2022-09-19,648.60,648.60,"2,770.45",512.05,"3,704.25",558.49,"15,569.02","2,606.29",0.00
53,69,Sergio Alexis Bautista Ángeles,"31,887.10",1500,2022-12-01,"1,110.95","1,121.81","5,919.29",911.26,"6,041.92",956.61,"25,500.11","1,275.48",0.00


In [7]:
aumento_2023 = 1.09

In [8]:
def vales_2023(vale):
    if (vale * aumento_2023) >= 1500:
        return 1500.0
    else:
        return (vale * aumento_2023)

In [9]:
def imss_pt_2023(x, date):
    nomina = CalculoIMSSPatron()
    nomina.set_fecha_base(date)
    nomina.set_sdi(x)
    nomina._uma = 96.22 * aumento_2023
    return nomina.total_imss()

In [10]:
def imss_ob_2023(x):
    nomina = CalculoIMSSObrero()
    nomina.set_sdi(x)
    nomina._uma = 96.22 * aumento_2023
    return nomina.total_imss()

In [11]:
date_2023 = pd.to_datetime('2023/12/31')

In [12]:
df['Sueldo 2023'] = df['Sueldo mensual'].apply(lambda x: x * 1)
df['Vales 2023'] = df['Vales'].apply(lambda x: x * 1 )

In [13]:
df['SDI_2023'] = df.apply(lambda x: SBC.funcion_sdi(sueldo = x['Sueldo 2023'], a = x['Fecha Ingreso'], b = date_2023), axis=1)
df['SBC_2023'] = df.apply(lambda x: SBC.funcion_sbc(sueldo = x['Sueldo 2023'], vales = x['Vales 2023'], a = x['Fecha Ingreso'], b = date_2023), axis=1)
df['ISR_2023'] = (df['SDI_2023'] * 30.4).apply(lambda x: ISR.function_isr(x))
df['IMSS_O_2023'] = df['SBC_2023'].apply(lambda x: imss_ob_2023(x))
df['IMSS_P_2023'] = df.apply(lambda x: imss_pt_2023(x = x['SBC_2023'], date = date_2023), axis=1)
df['ISN_2023'] = df['Sueldo 2023'] * 0.03
_isr_ = df['Sueldo 2023'].apply(lambda x: ISR.function_isr(x))
df['Neto 2023'] = df['Sueldo 2023'] - _isr_ - df['IMSS_O_2023']
df['Aguinaldo_2023'] = df.apply(lambda x: SBC.funcion_aguinaldo(sueldo = x['Sueldo 2023'], a = x['Fecha Ingreso'], b = date_2023), axis=1)
df['Prima Vacacional_2023'] = df.apply(lambda x: SBC.funcion_pv(sueldo = x['Sueldo 2023'], date_ingreso = x['Fecha Ingreso'], date_base = date_2023), axis=1)

In [ ]:
df.tail()

In [ ]:
aumento_2024 = 1.18

In [ ]:
def vales_2024(vale):
    if (vale * aumento_2024) > 1500:
        return 1500.0
    else:
        return (vale * aumento_2024)

In [ ]:
def imss_pt_2024(x, date):
    nomina = CalculoIMSSPatron()
    nomina.set_fecha_base(date)
    nomina.set_sdi(x)
    nomina._uma = 96.22 * aumento_2024
    return nomina.total_imss()

In [ ]:
def imss_ob_2024(x):
    nomina = CalculoIMSSObrero()
    nomina.set_sdi(x)
    nomina._uma = 96.22 * aumento_2024
    return nomina.total_imss()

In [ ]:
date_2024 = pd.to_datetime('2024/12/31')

In [ ]:
df['Sueldo 2024'] = df['Sueldo mensual'].apply(lambda x: x * aumento_2024)
df['Vales 2024'] = df['Vales'].apply(lambda x: vales_2024(x) )

In [ ]:
df['SDI_2024'] = df.apply(lambda x: SBC.funcion_sdi(sueldo = x['Sueldo 2024'], a = x['Fecha Ingreso'], b = date_2024), axis=1)
df['SBC_2024'] = df.apply(lambda x: SBC.funcion_sbc(sueldo = x['Sueldo 2024'], vales = x['Vales 2024'], a = x['Fecha Ingreso'], b = date_2024), axis=1)
df['ISR_2024'] = (df['SDI_2024'] * 30.4).apply(lambda x: ISR.function_isr(x))
df['IMSS_O_2024'] = df['SBC_2024'].apply(lambda x: imss_ob_2024(x))
df['IMSS_P_2024'] = df.apply(lambda x: imss_pt_2024(x = x['SBC_2024'], date = date_2024), axis=1)
df['ISN_2024'] = df['Sueldo 2024'] * 0.03
_isr_ = df['Sueldo 2024'].apply(lambda x: ISR.function_isr(x))
df['Neto 2024'] = df['Sueldo 2024'] - _isr_ - df['IMSS_O_2024']
df['Aguinaldo_2024'] = df.apply(lambda x: SBC.funcion_aguinaldo(sueldo = x['Sueldo 2024'], a = x['Fecha Ingreso'], b = date_2024), axis=1)
df['Prima Vacacional_2024'] = df.apply(lambda x: SBC.funcion_pv(sueldo = x['Sueldo 2024'], date_ingreso = x['Fecha Ingreso'], date_base = date_2024), axis=1)

In [ ]:
df

In [14]:
df.to_excel('./resultados/proyeccion2023p.xlsx', index=False, sheet_name='proyeccion')

In [ ]:
df_t = df.copy(deep=True)

In [ ]:
df_t.T.head(14)

In [ ]:
df_t = df_t.T

In [ ]:
df_t.to_excel('./resultados/proyeccion_t.xlsx', index=True, sheet_name='proyeccion')